# Notebook 08: Reporting and Publication Tools

**New in v0.10.0**: Comprehensive reporting tools for generating publication-ready analysis

This notebook demonstrates:
1. Creating comprehensive model reports
2. Generating summary statistics and evaluation metrics
3. Exporting to multiple formats (Markdown, LaTeX, figures)
4. Comparing multiple models
5. Creating publication-quality visualizations

**Use Case**: Preparing results for academic papers, reports, or presentations

In [ ]:
# Setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from epydemics import DataContainer, Model
from epydemics.analysis import ModelReport, create_comparison_report

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("Epydemics v0.10.0 - Reporting Tools Demo")
print("="*60)

## 1. Prepare Sample Data

Using realistic measles data (annual incidence)

In [ ]:
# Create realistic measles data (Mexico 2010-2024)
np.random.seed(42)
dates = pd.date_range('2010', periods=15, freq='YE')

# Incident cases per year (realistic pattern)
incident_cases = np.array([
    220, 55, 667, 164, 81,   # 2010-2014: sporadic
    34, 12, 0, 0, 4,         # 2015-2019: near elimination
    18, 45, 103, 67, 89      # 2020-2024: reintroduction
])

# Cumulative deaths (CFR ~0.1%)
incident_deaths = np.array([1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 1])
cumulative_deaths = np.cumsum(incident_deaths)

# Population
population = [120_000_000 + i*2_000_000 for i in range(15)]

# Create DataFrame
data = pd.DataFrame({
    'I': incident_cases,
    'D': cumulative_deaths,
    'N': population
}, index=dates)

print("Mexico Measles Data (2010-2024)")
print(data)

# Visualize the pattern
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(data.index, data['I'], 'o-', linewidth=2, markersize=6)
ax1.set_title('Incident Cases per Year', fontsize=12, fontweight='bold')
ax1.set_ylabel('Cases')
ax1.grid(True, alpha=0.3)

ax2.plot(data.index, data['D'], 'o-', linewidth=2, markersize=6, color='red')
ax2.set_title('Cumulative Deaths', fontsize=12, fontweight='bold')
ax2.set_ylabel('Deaths')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nData characteristics:")
print(f"  Total cases: {incident_cases.sum()}")
print(f"  Total deaths: {cumulative_deaths[-1]}")
print(f"  Case Fatality Rate: {cumulative_deaths[-1]/incident_cases.sum()*100:.2f}%")

## 2. Split Data and Train Model

Train on 2010-2019, test on 2020-2024

In [ ]:
# Split train/test
train_data = data.iloc[:10]  # 2010-2019 (10 years)
test_data = data.iloc[10:]   # 2020-2024 (5 years)

print(f"Training data: {train_data.index[0].year} - {train_data.index[-1].year} ({len(train_data)} years)")
print(f"Testing data:  {test_data.index[0].year} - {test_data.index[-1].year} ({len(test_data)} years)")

# Create model with incidence mode
container = DataContainer(train_data, mode='incidence', window=3)
model = Model(container)

print(f"\nModel mode: {model.mode}")
print(f"Frequency: {container.frequency}")

# Fit and forecast
model.create_model()
model.fit_model(max_lag=2)
model.forecast(steps=5)  # Forecast 5 years
model.run_simulations(n_jobs=1)
model.generate_result()

print("\n✅ Model fitted and forecast generated!")

## 3. Create ModelReport

The `ModelReport` class provides a high-level interface for analyzing results

In [ ]:
# Create comprehensive report
report = ModelReport(
    results=model.results,
    testing_data=test_data,
    compartments=['I', 'D'],
    model_name="Mexico Measles Forecast (2010-2024)"
)

print("✅ ModelReport created")
print(f"   Compartments: {report.compartments}")
print(f"   Has test data: {report.testing_data is not None}")

## 4. Generate Summary Statistics

Get comprehensive statistics for all compartments

In [ ]:
summary_df = report.generate_summary()

print("Summary Statistics:")
print("="*80)
display(summary_df)

print("\n📊 Key Insights:")
for _, row in summary_df.iterrows():
    print(f"  {row['Compartment']}:")
    print(f"    Mean forecast: {row['Mean']:.1f}")
    print(f"    Range: {row['Min']:.1f} - {row['Max']:.1f}")
    print(f"    Coefficient of Variation: {row['CV (%)']:.1f}%")

## 5. Evaluate Forecast Accuracy

Compare forecasts against test data with multiple metrics

In [ ]:
eval_df = report.get_evaluation_summary()

print("Forecast Evaluation Metrics:")
print("="*80)
display(eval_df)

# Highlight best performing method
for comp in ['I', 'D']:
    comp_eval = eval_df[eval_df['Compartment'].str.contains(comp)]
    best_mae = comp_eval.loc[comp_eval['MAE'].idxmin()]
    print(f"\n🏆 Best method for {comp}: {best_mae['Method']} (MAE: {best_mae['MAE']:.2f})")

## 6. Create Multi-Panel Visualization

Generate publication-quality figures with all compartments

In [ ]:
fig = report.plot_forecast_panel(
    figsize=(14, 8),
    save_path=None  # Don't save yet, just display
)

plt.show()

print("\n💡 Tip: Set save_path='output.png' and dpi=600 for publication quality!")

## 7. Export Reports

Generate reports in multiple formats for different purposes

In [ ]:
# Create output directory
output_dir = Path("notebook_outputs/reporting_demo")
output_dir.mkdir(parents=True, exist_ok=True)

# 1. Markdown report (for GitHub, documentation)
report.export_markdown(
    filepath=output_dir / "measles_report.md",
    include_summary=True,
    include_evaluation=True,
    include_figure=True
)
print(f"✅ Markdown report: {output_dir / 'measles_report.md'}")

# 2. LaTeX tables (for academic papers)
report.export_latex_table(
    filepath=output_dir / "table1_summary.tex",
    table_type="summary"
)
print(f"✅ LaTeX summary table: {output_dir / 'table1_summary.tex'}")

report.export_latex_table(
    filepath=output_dir / "table2_evaluation.tex",
    table_type="evaluation"
)
print(f"✅ LaTeX evaluation table: {output_dir / 'table2_evaluation.tex'}")

# 3. High-resolution figure (for publications)
fig_pub = report.plot_forecast_panel(
    figsize=(14, 10),
    save_path=output_dir / "figure1_forecast.png",
    dpi=300  # Use 600 for journal submission
)
plt.close(fig_pub)
print(f"✅ High-res figure: {output_dir / 'figure1_forecast.png'}")

print(f"\n📁 All outputs saved to: {output_dir}")

## 8. Preview Generated Markdown Report

In [ ]:
# Display the generated Markdown report
with open(output_dir / "measles_report.md", 'r') as f:
    markdown_content = f.read()

from IPython.display import Markdown
display(Markdown(markdown_content))

## 9. Model Comparison

Compare different model configurations side-by-side

In [ ]:
# Create alternative model with different smoothing window
container2 = DataContainer(train_data, mode='incidence', window=2)
model2 = Model(container2)
model2.create_model()
model2.fit_model(max_lag=2)
model2.forecast(steps=5)
model2.run_simulations(n_jobs=1)
model2.generate_result()

# Create third model with window=4
container3 = DataContainer(train_data, mode='incidence', window=4)
model3 = Model(container3)
model3.create_model()
model3.fit_model(max_lag=2)
model3.forecast(steps=5)
model3.run_simulations(n_jobs=1)
model3.generate_result()

print("✅ Created 3 models with different smoothing windows")

# Compare models
models = {
    "2-Year Window": model2.results,
    "3-Year Window (baseline)": model.results,
    "4-Year Window": model3.results
}

fig_comparison = create_comparison_report(
    models=models,
    testing_data=test_data,
    compartment='I',
    save_path=output_dir / "model_comparison.png"
)

plt.show()

print(f"\n✅ Comparison saved to: {output_dir / 'model_comparison.png'}")

## 10. Quantitative Model Comparison

In [ ]:
# Compare evaluation metrics across models
comparison_data = []

for name, results in models.items():
    temp_report = ModelReport(results, test_data, ['I'], name)
    eval_metrics = temp_report.evaluate()['I']['mean']
    
    comparison_data.append({
        'Model': name,
        'MAE': eval_metrics['mae'],
        'RMSE': eval_metrics['rmse'],
        'MAPE (%)': eval_metrics['mape'],
        'SMAPE (%)': eval_metrics['smape']
    })

comparison_df = pd.DataFrame(comparison_data)

print("Model Comparison - Incident Cases (I):")
print("="*80)
display(comparison_df)

# Highlight best model
best_model = comparison_df.loc[comparison_df['MAE'].idxmin()]
print(f"\n🏆 Best performing model: {best_model['Model']}")
print(f"   MAE: {best_model['MAE']:.2f}")
print(f"   RMSE: {best_model['RMSE']:.2f}")

## 11. Summary and Best Practices

### What We Learned

1. **ModelReport** provides one-stop reporting for model results
2. **Multiple export formats** support different use cases:
   - Markdown for documentation
   - LaTeX for academic papers
   - High-DPI PNG for publications
3. **Model comparison** tools make it easy to evaluate alternatives
4. **Automated metrics** ensure consistent reporting

### Best Practices for Publications

```python
# For journal submission
report.plot_forecast_panel(
    figsize=(14, 10),
    save_path="manuscript/figure1.png",
    dpi=600  # High resolution
)

# For LaTeX manuscripts
report.export_latex_table("manuscript/table1.tex", "summary")
report.export_latex_table("manuscript/table2.tex", "evaluation")

# For GitHub/documentation
report.export_markdown("docs/results.md")
```

### File Organization

```
project/
├── data/           # Input data
├── notebooks/      # Analysis notebooks
├── outputs/
│   ├── figures/    # PNG files (300-600 DPI)
│   ├── tables/     # LaTeX tables
│   └── reports/    # Markdown reports
└── manuscript/     # Final publication files
```

### Next Steps

- Explore other notebooks for specific use cases
- Read the [Reporting Guide](../../docs/REPORTING_GUIDE.md) for complete API reference
- Check [examples/reporting_example.py](../reporting_example.py) for scripted workflow

## 12. Files Generated

List all files created during this demo

In [ ]:
import os

print("Generated files:")
print("="*80)

for file in sorted(output_dir.glob('*')):
    size = os.path.getsize(file)
    if size < 1024:
        size_str = f"{size} B"
    elif size < 1024*1024:
        size_str = f"{size/1024:.1f} KB"
    else:
        size_str = f"{size/(1024*1024):.1f} MB"
    
    print(f"  📄 {file.name:40s} {size_str:>10s}")

print(f"\n📁 Location: {output_dir.absolute()}")
print("\n💡 These files are ready to use in your publications!")